In [1]:
 %pip install pydantic-ai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install requests
%pip install python-dotenv


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()   # just call it once

WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")
BASE_URL = os.getenv("BASE_URL")

def find_weather(city: str) -> dict:
    params = {
        'q': city,
        'appid': WEATHER_API_KEY,
        'units': "metric",
        'lang': 'en',
    }

    response = requests.get(BASE_URL, params=params)
    return response.json()



In [5]:
output = find_weather("london")
print(output)

{'cod': 401, 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}


Pydantic Ai Agent

In [6]:
import os
import requests
from pydantic import BaseModel
from pydantic_ai import Agent,RunContext
from pydantic_ai.settings import ModelSettings

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()
PPLX_API_KEY = os.getenv("PPLX_API_KEY")

In [8]:
class WeatherForecast(BaseModel):
    location: str
    description: str
    temperature_celsius: float

In [9]:
%pip install "pydantic-ai-slim[perplexity]"


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


In [10]:
weather_agent = Agent(
    model = "perplexity:sonar",
    model_settings = ModelSettings(temperature=0.0),
    output_type = str,
    system_prompt= "You are a helpful weather assistant. use the 'get_weather_forecast' tool to find current weather conditions for any city or place. provide clean and friendly answer."
)

ValueError: Unknown provider: perplexity

In [ ]:
@weather_agent.tool
def get_weather_forecast(ctx: RunContext, city:str) -> WeatherForecast:
    """
    Returns weather forecast for given city
    """
    url = BASE_URL,
    api_key = PPLX_API_KEY,
    params = {
        'q': city,
        'appid': WEATHER_API_KEY,
        'units': 'metric'
    }
    response = requests.get(url, params=params)
    result = response.json()

    return WeatherForecast(
        location = result['location']['name'],
        description = result['weather'][0]['description'].capitalize(),
        temperature_celsius = result['main']['temp'],

    )

In [ ]:
agent_response = weather_agent.run_sync("What is weather of chennai")
print(agent_response)

In [ ]:
question = input(" Ask About the Weather ")
result = weather_agent.run_sync(question)
print("\n Forecast: ", result.output)